In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import time
import datetime

nltk.download('words')
words = set(nltk.corpus.words.words())
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package words to
[nltk_data]     /Users/mosheborochov/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
df = pd.read_json('../data/mstream_datasets/united-2020-12-05-2020-12-15.json')
print(df['text'])
def stemming(words):
    ps=PorterStemmer()
    return [ps.stem(word) for word in words]

def remove_stop_words(words):
    return [c.lower() for c in words if c not in stop_words]

def lemmatizing(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]


def cleaner(tweet):
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    return tweet

def final_text(words):
     return ' '.join(words)

#df['text'] = df['text'].map(lambda x: cleaner(x))
#df['text'] = df['text'].apply(lambda x:word_tokenize(x))
#df['text'] = df['text'].apply(lambda x:remove_stop_words(x))
#df['text'] = df['text'].apply(lambda x: lemmatizing(x))
#df['text'] = df['text'].apply(lambda x: stemming(x))
#df['text'] = df['text'].apply(lambda x:final_text(x))

0        RT @DavidCornDC: Hey @United, a family member ...
1        RT @DavidCornDC: Hey @United, a family member ...
2        RT @DavidCornDC: Worse, @United, your flight a...
3        RT @DavidCornDC: Worse, @United, your flight a...
4        RT @DavidCornDC: Hey @United, a family member ...
                               ...                        
34638    RT @aviacionhr_info: Boeing 737-700 de @united...
34639    @CooperWoods04 @BBOZZY33 @united Just saw this...
34640    RT @aviacionhr_info: Boeing 737-700 de @united...
34641    RT @aviacionhr_info: Boeing 737-700 de @united...
34642    @Obsidian_Knee @TomiLahren @united Lol okay De...
Name: text, Length: 34643, dtype: object


In [3]:
#df = df.explode('hashtags').explode('mentions').explode('text')
df = df.explode('hashtags').explode('mentions')
continuous_index = []
#symbolic_index = ['hashtags', 'user_id', 'mentions', 'text']
symbolic_index = ['hashtags', 'mentions']

df_continuous = df.loc[:, continuous_index]
df_symbolic = df.loc[:, symbolic_index]
df_label = df.loc[:, ['is_anomaly']]

In [4]:
df_symbolic.loc[:,'hashtags'].unique()
#df_symbolic.loc[:,'user_id'].unique()
df_symbolic.loc[:,'mentions'].unique()
#df_symbolic.loc[:,'text'].unique()


hashtags_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'hashtags'].unique()):
    hashtags_dict[entry] = i
    
'''user_id_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'user_id'].unique()):
    user_id_dict[entry] = i'''

mentions_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'mentions'].unique()):
    mentions_dict[entry] = i+1
    
'''text_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'text'].unique()):
    text_dict[entry] = i'''

"text_dict = {}\nfor i, entry in enumerate(df_symbolic.loc[:,'text'].unique()):\n    text_dict[entry] = i"

In [5]:
df_symbolic

,hashtags,mentions
0,NaN,DavidCornDC
0,NaN,united
1,NaN,DavidCornDC
1,NaN,united
2,NaN,DavidCornDC
...,...,...
34641,NaN,united
34641,NaN,F1Caleb
34642,NaN,Obsidian_Knee
34642,NaN,TomiLahren


In [6]:
hashtags_dict

{nan: 0,
 'Fixit': 1,
 'aviation': 2,
 'AvGeek': 3,
 '737MAX': 4,
 'N27519': 5,
 'United': 6,
 'dontflyunited': 7,
 'BoycottUnitedAirlines': 8,
 'WearaDamnMask': 9,
 'friendlyskies': 10,
 'UnitedAirlines': 11,
 'Delta': 12,
 'Southwest': 13,
 'RenoTahoe': 14,
 'united': 15,
 'WearMaskProtectLife': 16,
 'NoFlyList': 17,
 'Airline101': 18,
 'Airlines': 19,
 'Travel': 20,
 'Reno': 21,
 'IAH': 22,
 'TuadaKuttaTommy': 23,
 'tuesdaymotivations': 24,
 'FlyUnitedGetCovid': 25,
 'CMH': 26,
 'WearAMask': 27,
 'cmh': 28,
 'GOPCowards': 29,
 'CovidVaccine': 30,
 'boycott': 31,
 'Copa': 32,
 'shame': 33,
 'deltafliesright': 34,
 'DontFlyUnited': 35,
 'UnitedAirlinesComplicit': 36,
 'BoycottUnited': 37,
 'Greedy': 38,
 'boycottUnited': 39,
 'Israel': 40,
 'UnitedNoMask': 41,
 'Duh': 42,
 'TrumpShutdown': 43,
 'Christmas': 44,
 'COVIDIOTS': 45,
 'Shame': 46,
 'unitedairlinesnotsafe': 47,
 'SFO': 48,
 'OnwardandUpward': 49,
 'airline': 50,
 'CustomerService': 51,
 'unitedAIRLINES': 52,
 'poorcustomere

In [7]:
df_symbolic.loc[:,'hashtags'] =  df_symbolic.loc[:,'hashtags'].map(hashtags_dict)
#df_symbolic.loc[:,'user_id'] =  df_symbolic.loc[:,'user_id'].map(user_id_dict)
df_symbolic.loc[:,'mentions'] =  df_symbolic.loc[:,'mentions'].map(mentions_dict)
#df_symbolic.loc[:,'text'] =  df_symbolic.loc[:,'text'].map(text_dict)
#df_symbolic = df_symbolic.fillna(0)
df_symbolic

,hashtags,mentions
0,0,1
0,0,2
1,0,1
1,0,2
2,0,1
...,...,...
34641,0,2
34641,0,4531
34642,0,3516
34642,0,2707


In [8]:
df_continuous.to_csv("CentralParkNYCnumeric.txt", index=False, header=False)
df_symbolic.to_csv("CentralParkNYCcateg.txt", index=False, header=False)
df_label.to_csv("CentralParkNYC_label.txt", index=False, header=False)
def create_unix(x):
    return int(time.mktime((x).timetuple()))
df.loc[:,'created_at'].map(create_unix).to_csv("CentralParkNYCtime.txt", index=False, header=False)

In [9]:
df_symbolic.shape[0]

93819

In [10]:
text_file = open("CentralParkNYCnumeric.txt", "w")
df_symbolic.shape[0]
n = text_file.write('\n'*df_symbolic.shape[0])
text_file.close()

In [11]:
#df_label.groupby(['is_anomaly']).agg(['min', 'max', 'count', 'nunique'])
#df_label.groupby(['is_anomaly']).agg(['min', 'max', 'count', 'nunique'])
df['is_anomaly'].value_counts()

0    59939
1    33880
Name: is_anomaly, dtype: int64

In [12]:
df.shape

(93819, 15)

In [13]:
df_symbolic

,hashtags,mentions
0,0,1
0,0,2
1,0,1
1,0,2
2,0,1
...,...,...
34641,0,2
34641,0,4531
34642,0,3516
34642,0,2707


In [14]:
df_symbolic.iloc[1]

hashtags    0
mentions    2
Name: 0, dtype: int64

In [131]:
score = pd.read_csv('score.txt', header=None)